# Library Mangament System 📖🤓
## an Objeact-Oriented Programming Project

##### import statements

In [1]:
import datetime
import random
import uuid

In [2]:
class Library:

    __books = []
    __orders = []
    index = 0

    def add_book(self, name, auther):
        self.index += 1
        book = Book(id=self.index, name=name, auther=auther)
        self.__books.append(book)
        return True

    def delete_book(self, name):
        for book in self.__books:
            if book.name == name:
                print(f"{book} is deleted")
                self.__books.remove(book)
                book.delete()
                return True
        print("Book is Not Founded")
        return False

    def display_books(self):
        return self.__books

    def display_available_book(self):
        available_books = [book for book in self.__books if book.check_availability()]
        for book in available_books:
            print(book)

    def search(self, book_name):
        for book in self.__books:
            if book.name == book_name:
                return book

    def add_order(self, new_order):
        self.__orders.append(new_order)

    def check_order_status(self):
        order_id = input("You want to check the order status : ..")
        for order in self.__orders:
            if order.id == order_id:
                return f"The Order status is {order.get_status()} and Due Date {order.due_date}"
        return "No Order with Such Name"

    def search_order_by_id(self):
        order_id = input("Please Enter order id: .. ")
        for order in self.__orders:
            if order.id == order_id:
                return order

    def rate_user(self, order):
        # rate the user
        ans = datetime.datetime.now() - order.due_date
        if ans.days < 0:
            print("You are late")
            order.borrower.set_rate(order.borrower.get_rate() + 2 * -2)
        else:
            print("You returned the book just in time")
            order.borrower.set_rate(order.borrower.get_rate() + 2 * 2)

    def retrieve_book(self):
        order = self.search_order_by_id()
        if order:
            # change the status of the book
            order.set_status("Finished")
            # change the book availability
            order.book.set_availability(True)
            # remove order from the orders list
            self.__orders.remove(order)
            self.rate_user(order=order)
            print(f"Your points now is ... {order.borrower.get_rate()}")
            print("Nice!! Done!!")
            order.delete_order()
        else:
            print("No Order with such name")

    def display_orders(self):
        print(self.__orders)

In [3]:
class Book:
    def __init__(self, id, name, auther, is_available=True):
        self.id = id
        self.name = name
        self.auther = auther
        self.is_available = is_available

    def __repr__(self):
        return f"BookName:{self.name}.Auther:{self.auther}"

    def set_availability(self, new_val):
        self.is_available = new_val

    def check_availability(self):
        return self.is_available

    def delete(self):
        del self

In [4]:
class Reader(Library):
    def __init__(self, name, rate=5):
        self.id = uuid.uuid4()
        self.name = name
        self.rate = rate

    def __repr__(self):
        return f"User {self.name}, {self.rate}"

    def set_rate(self, new_rate):
        self.rate = new_rate

    def get_rate(self):
        return self.rate

    def request_book(self, name):
        book = self.search(name)
        if book:
            # There is a book with such name
            # check availability
            if book.check_availability():
                # create a borrow order
                b = BorrowOrder(book=book, reader=self)
                # make book not available
                book.set_availability(False)
                # add the book to the orders
                self.add_order(b)
                print(f"Here is the order Number memorize it,. Ok? {b.id}")
            else:
                # Book is Already Borrowed
                print("Book is Not Available")
        else:
            print("No book with such name")

    def return_book(self):
        self.retrieve_book()

In [5]:
class BorrowOrder:
    def __init__(self, book, reader):
        self.id = book.name + '-' + reader.name + str(random.randint(1, 11))
        self.book = book
        self.borrower = reader
        self.due_date = datetime.datetime.today() - datetime.timedelta(days=5)
        self.status = "Borrowed"

    def __repr__(self):
        return f"Order {self.book}, {self.borrower}, {self.due_date}, {self.status}"

    def set_status(self, new_status):
        self.status = new_status

    def get_status(self):
        return self.status

    def delete_order(self):
        return self

In [6]:
# First Create the Library
lp = Library()

In [7]:
# add a number of books
lp.add_book("Book1", "A1")
lp.add_book("Book2", "A1")
lp.add_book("Book3", "A1")

True

In [8]:
# create a number of readers
user1 = Reader("Rana")
user2 = Reader("Koko")
user3 = Reader("Salem")

In [9]:
# search the books
print(lp.display_books())

[BookName:Book1.Auther:A1, BookName:Book2.Auther:A1, BookName:Book3.Auther:A1]


#### user1 searches for a book that is not in the library and user2 searched for a book that is in the library

In [10]:
# user1 wants to search for a book
book = user1.search("Koko")
if book:
    print(book)
else:
    print("No book with such name")

No book with such name


In [11]:
book = user2.search("Book3")
if book:
    print(f"Here is The book you asked about {book}")
else:
    print("No book with such name")

Here is The book you asked about BookName:Book3.Auther:A1


## Borrowing request
### <li>user1 requests book3 First and borrow it</li>
### <li>later user2 requests book3 but it is not available for borrowing</li>

In [12]:
# user2 decided to borrow the book
user2.request_book("Book3")

Here is the order Number memorize it,. Ok? Book3-Koko3


In [13]:
# user3 decided to borrow the same book
user3.request_book("Book3") # shows that the book is not available for borrowing

Book is Not Available


### Library checks for the order status

In [14]:
print(lp.check_order_status())

You want to check the order status : ..
No Order with Such Name


### user1 => Returning the book !!!!!

In [15]:
# user2 decided to return the book
user2.retrieve_book()

Please Enter order id: .. Book3-Koko3
You returned the book just in time
Your points now is ... 9
Nice!! Done!!


### user3 now can request the book that was returned for borrowing

In [16]:
# user3 now can request the book
user3.request_book("Book3")

Here is the order Number memorize it,. Ok? Book3-Salem1


### Each user has a point that is updated if they returned the book borrowed before the due date

In [17]:
user1.get_rate()

5

In [18]:
user2.get_rate()

9

In [19]:
user1.request_book("Book1")

Here is the order Number memorize it,. Ok? Book1-Rana10


### user1 Returned the book on time => points increased.

In [20]:
user1.return_book()

Please Enter order id: .. Book1-Rana10
You returned the book just in time
Your points now is ... 9
Nice!! Done!!


# Rana(2023)